In [3]:
from collections import deque
from collections import defaultdict
from heapdict import heapdict

In [16]:
class Graph:
    def __init__(self, oriented=True):
        self.edges = 0
        self.nodes = deque()
        self.adj_list = defaultdict(list)
        self.adj_list_weight = defaultdict(list)
        self.dist = heapdict()
        self.prev = defaultdict()
        self.degree = defaultdict()
        self.infinity = float('inf')
        self.oriented = bool(oriented)
        self.color = defaultdict(int) # dict for vertex coloring -1 and 1
        self.adj_matrix = list()
        
    def add_node(self, name):
        self.nodes.append(name)
        
    def add_edge(self, adj, weight=None):
        self.adj_list[adj[0]].append(adj[1])
        if weight:
            self.adj_list_weight[adj[0]].append([adj[1], weight])
        # if undirected graph use next line
        if not self.oriented:
            self.adj_list[adj[1]].append(adj[0])
        self.edges += 1
    
    def create_adj_matrix(self):
        self.adj_matrix = [[0 for column in range(len(self.nodes))]
                           for line in range(len(self.nodes))]
        if not self.adj_list_weight:
            for node in self.nodes:
                for neigh in self.adj_list[node]:
                    self.adj_matrix[node - 1][neigh - 1] = 1
        else:
            for node in self.nodes:
                for neigh, weight in self.adj_list[node]:
                    self.adj_matrix[node - 1][neigh - 1] = weight
    
    def dfs(self, node, path=None):
        if not path:
            path = set()
        path.add(node)
        for neighbour in self.adj_list[node]:
            if neighbour not in path:
                self.dfs(neighbour, path)
        return path
    
    def init_weights(self, start=1):
        for node in self.nodes:
            self.dist[node] = self.edges + 1
        self.dist[start] = 0
        
    def init_weights_dijkstra(self, start=1):
        for node in self.nodes:
            self.dist[node] = self.infinity
            self.prev[node] = None
        self.dist[start] = 0
    
    def bfs(self, start=1):
        query = deque()
        query.append(start)
        self.init_weights()
        while len(query):
            node = query.popleft()
            for neigh in self.adj_list[node]:
                if self.dist[neigh] == self.edges + 1:
                    query.append(neigh)
                    self.dist[neigh] = self.dist[node] + 1
    
    def dijkstra(self, start=1):
        self.init_weights_dijkstra(start)
        heap = heapdict()
        for node in self.nodes:
            heap[node] = self.dist[node]
        
        while len(heap):
            node = heap.popitem()[0]
            for neigh, weight in self.adj_list_weight[node]:
                if self.dist[neigh] > self.dist[node] + weight:
                    self.dist[neigh] = self.dist[node] + weight
                    self.prev[neigh] = node
                    heap[neigh] = self.dist[neigh]
                    
    def update_weights(self, node, edge):
        if self.dist[edge[0]] > self.dist[node] + edge[1]:
            self.dist[edge[0]] = self.dist[node] + edge[1]
            return 1
        return 0

    def bellman_ford(self, start=1):
        self.init_weights_dijkstra(start)
        
        for _ in range(len(self.nodes) - 1):
            s = 0
            for node, edges_list in self.adj_list_weight.items():
                for edge in edges_list:
                    s += self.update_weights(node, edge)
            if s == 0:
                break
        
    def find_subgraphs(self):
        pathes = list()
        visited_nodes = set()
        for node in self.nodes:
            if node not in visited_nodes:
                pathes.append(self.dfs(node))
                visited_nodes |= pathes[-1]
        return pathes
    
    def find_nodes_degree(self):
        for node in self.nodes:
            self.degree[node] = len(self.adj_list[node])
    
    def color_graph(self, node=1, path=None, parent_color=-1):
        if not path:
            path = set()
        path.add(node)
        self.color[node] = -1 * parent_color
        for neighbour in self.adj_list[node]:
            if neighbour not in path:
                self.color_graph(neighbour, path, self.color[node])
    
    def isBipartite(self):
        subg = [i.pop() for i in self.find_subgraphs()]
        for node in subg:
            self.color_graph(node)
            
        for node in self.nodes:
            for neigh in self.adj_list[node]:
                if self.color[node] == self.color[neigh]:
                    return False
        return True
    
    def isCyclicUtil(self, v, visited, recStack): 
  
        visited[v] = True
        recStack[v] = True
  
        for neighbour in self.adj_list[v]:
            if visited[neighbour] == False: 
                if self.isCyclicUtil(neighbour, visited, recStack): 
                    return True
            elif recStack[neighbour] == True: 
                return True

        recStack[v] = False
        return False
  
    # Returns true if graph is cyclic else false 
    def isCyclic(self): 
        visited = defaultdict(bool)
        recStack = defaultdict(bool)
        for node in self.nodes: 
            if visited[node] == False: 
                if self.isCyclicUtil(node,visited,recStack): 
                    return True
        return False
    
    def __repr__(self):
        if self.adj_list_weight:
            output = "Graph adj list:\ni.e `>node: edges[[node, weight], ...]`\n"
            for node in self.nodes:
                output += f">{node}: {self.adj_list_weight[node]}\n"
        else:
            output = "Graph adj list:\ni.e `>node: edges[node, ...]`\n"
            for node in self.nodes:
                output += f">{node}: {self.adj_list[node]}\n"
        return output

In [19]:
with open('./example.txt', 'r') as f:
    graphs_num = int(f.readline())
    for _i in range(graphs_num):
        f.readline()
        g = Graph(oriented=0)
        n, e = map(int, f.readline().split())
        for i in range(1, n + 1):
            g.add_node(i)
        for _ in range(e):
            ar = [int(_) for _ in f.readline().split()]
            g.add_edge(ar[:2])
        g.create_adj_matrix()
        print(*g.adj_matrix, sep='\n', end ='\n' + '-' * 10 + '\n')


[0, 1, 1, 0]
[1, 0, 1, 1]
[1, 1, 0, 1]
[0, 1, 1, 0]
----------
[0, 1, 0, 1]
[1, 0, 0, 1]
[0, 0, 0, 1]
[1, 1, 1, 0]
----------


In [207]:
g

Graph adj list:
i.e `>node: edges[node, ...]`
>1: [779, 318, 953, 776]
>2: [238, 106]
>3: []
>4: [277, 747, 719]
>5: [152, 627, 324]
>6: [575, 903]
>7: [451]
>8: [448, 236]
>9: [412, 743]
>10: [371, 328]
>11: [405, 242, 783]
>12: [489, 780, 122]
>13: []
>14: [725, 939, 308, 251, 116]
>15: [323]
>16: [404, 51]
>17: [924, 490]
>18: [631]
>19: [89]
>20: []
>21: [543, 643, 587]
>22: []
>23: [348]
>24: [648]
>25: [295, 670]
>26: [687]
>27: [929, 57, 267, 348]
>28: [626, 185, 373, 295]
>29: [893]
>30: [238]
>31: [148]
>32: [783, 473, 780, 448]
>33: []
>34: [635]
>35: []
>36: [370, 902]
>37: [583]
>38: [211, 962, 687, 541]
>39: [236, 135, 256]
>40: [378]
>41: [408]
>42: [156, 347]
>43: [938, 252, 832, 212, 478, 659, 75]
>44: []
>45: [248, 388, 684, 753]
>46: [667]
>47: [266, 255]
>48: []
>49: [297, 350, 73]
>50: [837, 197]
>51: [340, 688, 16, 478, 405]
>52: [781, 125]
>53: [884, 602]
>54: []
>55: [136, 259]
>56: [566]
>57: [509, 27, 911]
>58: [877, 938, 925]
>59: [665]
>60: [174, 361, 582, 55

In [58]:
graphs[0].find_cycle()

[1, 2, 3, 1]